# Avance 3 — Transformación de datos con Python

Este notebook ejecuta todas las consignas del Avance 3:
- Carga de CSVs y preparación básica
- Cálculo de TotalPriceCalculated
- Detección de outliers (IQR) y columna IsOutlier
- Feature SaleHour y hora pico de ventas
- Clasificación Entre semana vs Fin de semana y comparación de ventas
- Cálculo de AgeAtHire y YearsExperience
- Dataset final listo para modelado (con transformaciones) y export a CSV

Notas:
- Este notebook asume que los CSV están en `../data/` relativos a este archivo.
- La variable objetivo es TotalPriceCalculated (sin transformaciones).

In [2]:
# 1) Imports y configuración
import os
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 160)

# Paths: detectar raíz del proyecto buscando carpeta data/
cwd = Path.cwd()
ROOT = None
if (cwd / 'data').exists():
    ROOT = cwd
elif (cwd.parent / 'data').exists():
    ROOT = cwd.parent
else:
    p = cwd
    for _ in range(4):
        if (p / 'data').exists():
            ROOT = p
            break
        if p.parent == p:
            break
        p = p.parent
if ROOT is None:
    raise FileNotFoundError(f'No se encontró carpeta data/ desde {cwd}')

DATA_DIR = ROOT / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print('CWD:', cwd)
print('ROOT:', ROOT)
print('DATA_DIR:', DATA_DIR)
print('Archivos en data/:', sorted([p.name for p in DATA_DIR.glob('*.csv')]))

CWD: c:\Users\CTI23994\Dropbox\Data Engineering - HENRY\proyecto_integrador\notebooks
ROOT: c:\Users\CTI23994\Dropbox\Data Engineering - HENRY\proyecto_integrador
DATA_DIR: c:\Users\CTI23994\Dropbox\Data Engineering - HENRY\proyecto_integrador\data
Archivos en data/: ['categories.csv', 'cities.csv', 'countries.csv', 'customers.csv', 'employees.csv', 'products.csv', 'sales.csv']


In [3]:
# 2) Carga de CSVs con dtypes y fechas
parse_dates_sales = ['SalesDate']
parse_dates_employees = ['BirthDate', 'HireDate']

def read_csv_safe(path, **kwargs):
    if not path.exists():
        raise FileNotFoundError(f'No existe: {path}')
    return pd.read_csv(path, **kwargs)

sales = read_csv_safe(DATA_DIR / 'sales.csv', parse_dates=parse_dates_sales, dayfirst=False, infer_datetime_format=True)
products = read_csv_safe(DATA_DIR / 'products.csv')
customers = read_csv_safe(DATA_DIR / 'customers.csv')
employees = read_csv_safe(DATA_DIR / 'employees.csv', parse_dates=parse_dates_employees, dayfirst=False, infer_datetime_format=True)
categories = read_csv_safe(DATA_DIR / 'categories.csv')
cities = read_csv_safe(DATA_DIR / 'cities.csv')
countries = read_csv_safe(DATA_DIR / 'countries.csv')

print('sales:', sales.shape)
print('products:', products.shape)
print('customers:', customers.shape)
print('employees:', employees.shape)
print('categories:', categories.shape)
print('cities:', cities.shape)
print('countries:', countries.shape)

display(sales.head(3))

C:\Users\CTI23994\AppData\Local\Temp\ipykernel_26516\1102880518.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(path, **kwargs)


sales: (6758125, 9)
products: (452, 9)
customers: (98759, 6)
employees: (23, 8)
categories: (11, 2)
cities: (96, 4)
countries: (206, 3)


C:\Users\CTI23994\AppData\Local\Temp\ipykernel_26516\1102880518.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(path, **kwargs)


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0


In [4]:
# 3) Preparación: unir UnitPrice a sales y normalizar Discount
# Adaptado a nombres reales del CSV (CamelCase)

# Detectar columnas de clave y precio
prod_key = 'ProductID' if 'ProductID' in products.columns else ('product_id' if 'product_id' in products.columns else None)
sales_prod_key = 'ProductID' if 'ProductID' in sales.columns else ('product_id' if 'product_id' in sales.columns else None)
price_cols = [c for c in ['UnitPrice','Price','Unit_Price','unit_price','price'] if c in products.columns]
if not price_cols:
    raise KeyError('No se encontró columna de precio unitario en products (UnitPrice/Price).')
price_col = price_cols[0]

if sales_prod_key is None or prod_key is None:
    raise KeyError('No se encontró columna ProductID/product_id para unir sales y products.')

if 'UnitPrice' not in sales.columns:
    sales = sales.merge(products[[prod_key, price_col]].rename(columns={prod_key: sales_prod_key, price_col: 'UnitPrice'}),
                        on=sales_prod_key, how='left', validate='many_to_one')

# Normalizar Discount si está en porcentaje entero (e.g., 10 -> 0.10)
if 'Discount' in sales.columns:
    disc = sales['Discount']
    if disc.max() > 1.0:
        sales['Discount'] = disc.clip(lower=0).astype(float) / 100.0
else:
    sales['Discount'] = 0.0  # si no existe, asumimos 0

sales[['Quantity','UnitPrice','Discount']].head(3)

,Quantity,UnitPrice,Discount
0,7,44.2337,0.0
1,7,62.5460,0.0
2,24,79.0184,0.0


In [5]:
# 4) PI2 — Calcular TotalPriceCalculated y detectar outliers (IQR)
sales['TotalPriceCalculated'] = (sales['Quantity'].astype(float) * sales['UnitPrice'].astype(float)) * (1.0 - sales['Discount'].astype(float))

q1 = sales['TotalPriceCalculated'].quantile(0.25)
q3 = sales['TotalPriceCalculated'].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

sales['IsOutlier'] = ((sales['TotalPriceCalculated'] < lower) | (sales['TotalPriceCalculated'] > upper)).astype(int)

outliers_count = int(sales['IsOutlier'].sum())
total_rows = int(len(sales))
print(f"Outliers: {outliers_count} de {total_rows} ({outliers_count/total_rows:.2%})")

display(sales[['Quantity','UnitPrice','Discount','TotalPriceCalculated','IsOutlier']].head(5))

Outliers: 48217 de 6758125 (0.71%)


,Quantity,UnitPrice,Discount,TotalPriceCalculated,IsOutlier
0,7,44.2337,0.0,309.63590,0
1,7,62.5460,0.0,437.82200,0
2,24,79.0184,0.0,1896.44160,0
3,19,81.3167,0.2,1236.01384,0
4,9,79.9780,0.0,719.80200,0


In [6]:
# 5) PI2 — Extraer hora y encontrar hora pico de ventas
if not pd.api.types.is_datetime64_any_dtype(sales['SalesDate']):
    sales['SalesDate'] = pd.to_datetime(sales['SalesDate'], errors='coerce')

sales['SaleHour'] = sales['SalesDate'].dt.hour
sales_by_hour = sales.groupby('SaleHour', dropna=False)['TotalPriceCalculated'].sum().sort_values(ascending=False)
display(sales_by_hour.head(10))
best_hour = int(sales_by_hour.index[0]) if len(sales_by_hour) else None
print('Hora con más ventas (TotalPriceCalculated):', best_hour)

SaleHour
16.0    1.790144e+08
20.0    1.789492e+08
2.0     1.784208e+08
6.0     1.783812e+08
19.0    1.783461e+08
0.0     1.783134e+08
17.0    1.782904e+08
9.0     1.781666e+08
11.0    1.781430e+08
15.0    1.780218e+08
Name: TotalPriceCalculated, dtype: float64

Hora con más ventas (TotalPriceCalculated): 16


In [7]:
# 6) PI3 — Entre semana vs fin de semana
sales['DayOfWeek'] = sales['SalesDate'].dt.dayofweek
sales['Weekend'] = sales['DayOfWeek'].isin([5,6]).astype(int)
sales['WeekCategory'] = np.where(sales['Weekend']==1, 'Fin de semana', 'Entre semana')

week_summary = sales.groupby('WeekCategory')['TotalPriceCalculated'].sum().sort_values(ascending=False)
display(week_summary)

WeekCategory
Entre semana     3.123405e+09
Fin de semana    1.192863e+09
Name: TotalPriceCalculated, dtype: float64

In [8]:
# 7) PI4 — Edad al contratar y años de experiencia al vender
# Detectar claves para join employees
sales_emp_key_candidates = [c for c in ['SalesPersonID','salesperson_id','EmployeeID','employee_id'] if c in sales.columns]
emps_key_candidates = [c for c in ['EmployeeID','employee_id','SalesPersonID','salesperson_id'] if c in employees.columns]
if not sales_emp_key_candidates or not emps_key_candidates:
    raise KeyError('No se encontraron columnas de join entre sales y employees (SalesPersonID/EmployeeID).')

sales_emp_key = sales_emp_key_candidates[0]
emps_key = emps_key_candidates[0]

cols_needed = [emps_key]
for c in ['BirthDate','HireDate']:
    if c not in employees.columns:
        raise KeyError(f'Columna faltante en employees: {c}')
    cols_needed.append(c)

sales_emp = sales.merge(employees[cols_needed].rename(columns={emps_key: sales_emp_key}),
                      on=sales_emp_key, how='left', validate='many_to_one')

# Calcular edad al contratar y experiencia al vender
def years_between(late, early):
    delta = (late - early).dt.days
    return np.floor(delta / 365.25)

sales_emp['AgeAtHire'] = years_between(sales_emp['HireDate'], sales_emp['BirthDate'])
sales_emp['YearsExperience'] = years_between(sales_emp['SalesDate'], sales_emp['HireDate'])

# Sanitizar valores negativos/imposibles
sales_emp.loc[sales_emp['AgeAtHire'] < 0, 'AgeAtHire'] = np.nan
sales_emp.loc[sales_emp['YearsExperience'] < 0, 'YearsExperience'] = np.nan

sales_emp[['BirthDate','HireDate','SalesDate','AgeAtHire','YearsExperience']].head(5)
print('Resumen AgeAtHire:\n', sales_emp['AgeAtHire'].describe())
print('Resumen YearsExperience:\n', sales_emp['YearsExperience'].describe())

Resumen AgeAtHire:
 count    6.758125e+06
mean     4.517497e+01
std      1.186272e+01
min      2.000000e+01
25%      4.000000e+01
50%      4.800000e+01
75%      5.400000e+01
max      6.500000e+01
Name: AgeAtHire, dtype: float64
Resumen YearsExperience:
 count    6.690599e+06
mean     4.021273e+00
std      2.102461e+00
min      0.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      6.000000e+00
max      8.000000e+00
Name: YearsExperience, dtype: float64
Resumen YearsExperience:
 count    6.690599e+06
mean     4.021273e+00
std      2.102461e+00
min      0.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      6.000000e+00
max      8.000000e+00
Name: YearsExperience, dtype: float64


In [9]:
# 8) Enriquecimiento opcional: categoría y ubicación cliente

df = sales_emp.copy()

# Categoría del producto
prod_key = 'ProductID' if 'ProductID' in products.columns else ('product_id' if 'product_id' in products.columns else None)
if prod_key is not None and 'ProductID' in df.columns:
    # agregar category_id
    if 'CategoryID' in products.columns:
        df = df.merge(products[[prod_key,'CategoryID']].rename(columns={prod_key:'ProductID'}), on='ProductID', how='left', validate='many_to_one')
        # traer nombre de categoría
        if {'CategoryID'}.issubset(df.columns) and {'CategoryID','CategoryName'}.issubset(categories.columns):
            df = df.merge(categories[['CategoryID','CategoryName']], on='CategoryID', how='left', validate='many_to_one')

# Ubicación del cliente (ciudad/país)
if 'CustomerID' in df.columns:
    if {'CustomerID','CityID'}.issubset(customers.columns):
        df = df.merge(customers[['CustomerID','CityID']], on='CustomerID', how='left', validate='many_to_one')
        if {'CityID','City','CountryID'}.issubset(cities.columns):
            df = df.merge(cities[['CityID','City','CountryID']], on='CityID', how='left', validate='many_to_one')
            if {'CountryID','Country'}.issubset(countries.columns):
                df = df.merge(countries[['CountryID','Country']], on='CountryID', how='left', validate='many_to_one')

# Mostrar muestra
cols_show = [c for c in ['ProductID','CategoryID','CategoryName','CustomerID','CityID','City','CountryID','Country'] if c in df.columns]
df[cols_show].head(3) if cols_show else df.head(3)
print('Columnas actuales (muestra):', cols_show if cols_show else '(ver head)')

Columnas actuales (muestra): ['ProductID', 'CategoryID', 'CategoryName', 'CustomerID', 'CityID']


In [11]:
# 9) Dataset final para modelado
# Usaremos columna objetivo 'TotalPriceCalculated'

target_col = 'TotalPriceCalculated'

# Selección base en nombres CamelCase del dataset real
base_cols_candidates = [
    'SalesID','SalesDate','ProductID','CustomerID','SalesPersonID',
    'Quantity','UnitPrice','Discount','SaleHour','Weekend',
    'IsOutlier', 'AgeAtHire','YearsExperience','CategoryName','Country'
]
base_cols = [c for c in base_cols_candidates if c in df.columns]

final_df = df[base_cols + ([target_col] if target_col in df.columns else [])].copy()

# One-Hot Encoding controlado para categóricas de baja cardinalidad
categoricas = []
for col in ['CategoryName','Country']:
    if col in final_df.columns:
        nunique = final_df[col].nunique(dropna=True)
        if nunique <= 30:
            categoricas.append(col)

if categoricas:
    final_df = pd.get_dummies(final_df, columns=categoricas, prefix=categoricas, drop_first=True)

# Guardar
out_path = PROCESSED_DIR / 'final_dataset.csv'
final_df.to_csv(out_path, index=False)
print('Dataset final guardado en:', out_path)
print('Shape final:', final_df.shape)
final_df.head(3)

Dataset final guardado en: c:\Users\CTI23994\Dropbox\Data Engineering - HENRY\proyecto_integrador\data\processed\final_dataset.csv
Shape final: (6758125, 24)


,SalesID,SalesDate,ProductID,CustomerID,SalesPersonID,Quantity,UnitPrice,Discount,SaleHour,Weekend,IsOutlier,AgeAtHire,YearsExperience,TotalPriceCalculated,CategoryName_Cereals,CategoryName_Confections,CategoryName_Dairy,CategoryName_Grain,CategoryName_Meat,CategoryName_Poultry,CategoryName_Produce,CategoryName_Seafood,CategoryName_Shell fish,CategoryName_Snails
0,1,2018-02-05 07:38:25.430,381,27039,6,7,44.2337,0.0,7.0,0,0,26.0,4.0,309.6359,False,True,False,False,False,False,False,False,False,False
1,2,2018-02-02 16:03:31.150,61,25011,16,7,62.5460,0.0,16.0,0,0,65.0,0.0,437.8220,False,False,False,True,False,False,False,False,False,False
2,3,2018-05-03 19:31:56.880,23,94024,13,24,79.0184,0.0,19.0,0,0,48.0,6.0,1896.4416,False,False,False,False,False,False,True,False,False,False


In [12]:
# 10) Resumen de resultados
from textwrap import dedent

summary = dedent(f'''\
- Outliers detectados: {outliers_count} sobre {total_rows} registros.
- Hora con mayor venta total: {best_hour}.
- Ventas por categoría de semana:\n{week_summary.to_string()}
- Columnas finales: {len(final_df.columns)} y filas: {final_df.shape[0]}.
Archivo exportado: {out_path}
''')
print(summary)

- Outliers detectados: 48217 sobre 6758125 registros.
- Hora con mayor venta total: 16.
- Ventas por categoría de semana:
WeekCategory
Entre semana     3.123405e+09
Fin de semana    1.192863e+09
- Columnas finales: 24 y filas: 6758125.
Archivo exportado: c:\Users\CTI23994\Dropbox\Data Engineering - HENRY\proyecto_integrador\data\processed\final_dataset.csv

